In [1]:
import Random

In [2]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [12]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()

    endstate,steps = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,maxsteps)

    println("Flux Balance on x")
    println(relative_error(calcstor.meanRpX,calcstor.meanRmX))

    println("Flux Balance on y")
    println(relative_error(calcstor.meanRpY,calcstor.meanRmY))

    println("Cov Balance on x,x")
    println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
    println("Cov Balance on y,y")
    println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
    println("Cov Balance on y,x")
    println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    println(steps)
    println(endstate)
    println(calcstor.meanx)
    println(calcstor.meany)


    return calcstor
end

balance_test (generic function with 1 method)

In [15]:
calcs = balance_test([10,50],[10.,1.,4.,1.,-1.,50.],10^7,10^10,Random.MersenneTwister())

Flux Balance on x
0.0006497384678714713
Flux Balance on y
0.0003582577495729137
Cov Balance on x,x
0.022995191812381962
Cov Balance on y,y
0.019856004496052293
Cov Balance on y,x
0.023520689079256987
100087578
[5, 27]
6.624907099147678
26.448908360587573


BalanceEqData(6.624907099147678, 26.448908360587573, 1.5136263069595476e6, 8.81916463327828e6, 9.500682254174376e7, 1.3668092596005922e7, 6.608646990910732, 6.612943674802119, 26.451774699208475, 26.46125464874981, 8.676208806340348e6, -1.1305538336904615e6, 1.4048130762351027e7, -7.981747644015707e6, 1.33186421763894e7, 3.470483522536139e7, 9.436410554640487e7, 5.619252304940411e7)

LoadError: MethodError: no method matching all!(::Vector{Bool})
[0mClosest candidates are:
[0m  all!(::AbstractArray, [91m::AbstractArray[39m; init) at /Applications/Julia-1.7 arm64.app/Contents/Resources/julia/share/julia/base/reducedim.jl:913
[0m  all!([91m::Function[39m, [91m::AbstractArray[39m, [91m::AbstractArray[39m; init) at /Applications/Julia-1.7 arm64.app/Contents/Resources/julia/share/julia/base/reducedim.jl:911

In [39]:
function reach_balance(initialstate,params,numsteps,maxsteps,rng,flux_thres,cov_thres,tries)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor = BalanceEqData()
    trynum = 0
    thresholdmet = false
    
    
    while (trynum < tries) && (!thresholdmet)
        react_steps = trunc(Int,numsteps*(tries-trynum)/tries)
        new_max = trunc(Int,maxsteps*(tries-trynum)/tries)
        statevector,steps = direct_gillespie!(statevector,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,react_steps,new_max)
        if (relative_error(calcstor.meanRpX,calcstor.meanRmX) < flux_thres) &&  (relative_error(calcstor.meanRpY,calcstor.meanRmY) < flux_thres)
            cbxx_comp = cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum) < cov_thres
            cbyy_comp = cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum) < cov_thres
            cbyx_comp = cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                            0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum) <cov_thres 
            thresholdmet = all([cbxx_comp,cbyy_comp,cbyx_comp])               
        end
        trynum += 1
        print("Balance Test for chained simulation ")
        println(trynum)
        println("Flux Balance on x")
        println(relative_error(calcstor.meanRpX,calcstor.meanRmX))
    
        println("Flux Balance on y")
        println(relative_error(calcstor.meanRpY,calcstor.meanRmY))
    
        println("Cov Balance on x,x")
        println(cov_balance_comp(calcstor.CXRmX/calcstor.weightsum,calcstor.CXRmX/calcstor.weightsum,
                                calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/calcstor.weightsum,calcstor.CXRpX/calcstor.weightsum))
        println("Cov Balance on y,y")
        println(cov_balance_comp(calcstor.CYRmY/calcstor.weightsum,calcstor.CYRmY/calcstor.weightsum,
                                calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/calcstor.weightsum,calcstor.CYRpY/calcstor.weightsum))
        println("Cov Balance on y,x")
        println(cov_balance_comp(calcstor.CYRmX/calcstor.weightsum,calcstor.CXRmY/calcstor.weightsum,
                                0,calcstor.CYRpX/calcstor.weightsum,calcstor.CXRpY/calcstor.weightsum))
    end
    println("Tries taken:")
    println(trynum)
return calcstor
end

reach_balance (generic function with 1 method)

In [41]:
calcs = reach_balance([10,50],[10.,1.,4.,1.,-1.,50.],10^5,10^6,Random.MersenneTwister(),0.01,0.02,5)

Balance Test for chained simulation 1
Flux Balance on x
0.008558661329839573
Flux Balance on y
0.0012700272020612477
Cov Balance on x,x
0.0008173211851171903
Cov Balance on y,y
0.01323324398396664
Cov Balance on y,x
0.040159143742127174
Balance Test for chained simulation 2
Flux Balance on x
0.004623337021760448
Flux Balance on y
0.0007944232726668928
Cov Balance on x,x
0.018420552639757753
Cov Balance on y,y
0.023810882792470484
Cov Balance on y,x
0.02193901663095724
Balance Test for chained simulation 3
Flux Balance on x
0.003739798586966899
Flux Balance on y
0.0004953766194218783
Cov Balance on x,x
0.02108819506399032
Cov Balance on y,y
0.018622607055314478
Cov Balance on y,x
0.024227953593445213
Balance Test for chained simulation 4
Flux Balance on x
0.0035068766407092373
Flux Balance on y
9.479810252751032e-5
Cov Balance on x,x
0.02474285786475098
Cov Balance on y,y
0.0210322867525137
Cov Balance on y,x
0.02377568222667025
Balance Test for chained simulation 5
Flux Balance on x
0.

BalanceEqData(6.639161547039104, 26.49253392235438, 45283.58103694541, 263588.40809664543, 2.8366864909866336e6, 407941.6046952185, 6.604830699006467, 6.627472696859598, 26.50989078743839, 26.50378805137718, 259339.2141736985, -33703.23752662457, 419250.4735216529, -238282.68776952696, 397596.39494839736, 1.037356856694794e6, 2.817643319762026e6, 1.6770018940866117e6)